In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max= 500,)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv, description="Query Arxiv Papers")
print(arxiv.name)

arxiv


In [9]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max= 500,)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [10]:
wiki.invoke("Attention is all you need")

'Page: Attention Is All You Need\nSummary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al. It is considered a foundational paper in modern artificial intelligence, and a main contributor to the AI boom, as the transformer approach has become the main architecture of a wi'

In [3]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [4]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily = TavilySearchResults()

In [6]:
tavily.invoke("Provide me the recent AI News")

[{'title': 'AI News | Latest AI News, Analysis & Events',
  'url': 'https://www.artificialintelligence-news.com/',
  'content': 'March 4, 2025\nAutoscience Carl: The first AI scientist writing peer-reviewed papers\nApplications\nMarch 3, 2025\nDeep Learning\nAlibaba Qwen QwQ-32B: Scaled reinforcement learning showcase\nArtificial Intelligence\nMarch 6, 2025\nDeepSeek-R1 reasoning models rival OpenAI in performance\nArtificial Intelligence\nJanuary 20, 2025\nNew AI training techniques aim to overcome current challenges\nDeep & Reinforcement Learning\nNovember 28, 2024 [...] AI News\nMarch 20, 2025\nUK minister in US to pitch Britain as global AI investment hub\n\nTelecoms Tech News\nMarch 20, 2025\nOfcom plans rules to drive UK full-fibre broadband to 96% coverage\n\nMarketing Tech News\nMarch 20, 2025\nOracle continues talks with White House to run TikTok\n\nMarketing Tech News\nMarch 19, 2025\nSocial media platforms face fines and criminal prosecution in UK\n\nAI News\nMarch 19, 2025\

In [11]:
tools = [arxiv, wiki, tavily]

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="qwen-qwq-32b")

In [13]:
llm.invoke("what is AI")

AIMessage(content='\n<think>\nOkay, the user is asking, "What is AI?" I need to provide a clear and comprehensive explanation. Let me start by defining AI in simple terms. AI stands for Artificial Intelligence, right? So, I should explain that it\'s about making machines that can perform tasks requiring human-like intelligence. But wait, maybe I should break it down more.\n\nFirst, I should mention that AI refers to the simulation of human intelligence in machines. But intelligence is a broad term. What aspects of intelligence are we talking about? Like, things like learning, problem-solving, perception, and language understanding. Maybe I should list those elements.\n\nThen, maybe categorize AI into types. There\'s the distinction between narrow or weak AI and general or strong AI. Narrow AI is what we have now, like voice assistants or recommendation systems. They\'re good at specific tasks but lack general intelligence. General AI, or AGI, is hypothetical, where a machine could hand

In [15]:
llm_with_tools = llm.bind_tools(tools=tools)

In [16]:
llm_with_tools.invoke("What is the recent news on AI?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mgyp', 'function': {'arguments': '{"query": "recent news on AI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 245, 'prompt_tokens': 356, 'total_tokens': 601, 'completion_time': 0.609613572, 'prompt_time': 0.054247748, 'queue_time': 0.386715974, 'total_time': 0.66386132}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_512a3da6bb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-14126f06-3f0d-4f84-b72c-a452ad27b677-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent news on AI'}, 'id': 'call_mgyp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 356, 'output_tokens': 245, 'total_tokens': 601})

In [17]:
llm_with_tools.invoke("What is the latest research on Quantum Computing?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n0ap', 'function': {'arguments': '{"query": "Quantum Computing"}', 'name': 'arxiv'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_tokens': 357, 'total_tokens': 517, 'completion_time': 0.39967818, 'prompt_time': 0.045403762, 'queue_time': 0.16179084700000002, 'total_time': 0.445081942}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_512a3da6bb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5f6d28d6-8d5f-48f2-8b64-08219e6a177d-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'Quantum Computing'}, 'id': 'call_n0ap', 'type': 'tool_call'}], usage_metadata={'input_tokens': 357, 'output_tokens': 160, 'total_tokens': 517})

## Workflow 

In [18]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages